<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/AnswerClusterering_RougeL_Llama8b_Xquad_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##QA pairs generated using llama8b on xquad

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import json
import pandas as pd

file_path = '/content/drive/My Drive/generated_answers_Llama3.1-8b-xquad.json'

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.DataFrame(data)
print(df.head())

   question_id                                           question  \
0          320                             ما هو المرفق بالسواكن؟   
1         1007     ما الدليل الذي تقدمه الجينات المُتَبَرَّع بها؟   
2          605  ماذا حدث في عام 1992 ضمن إجراءٍ على نطاق الممل...   
3          425                   متى أحدِثت توجيهية مجلس الأعمال؟   
4            4  من سجل أعلى عدد من الاستحواذات للفريق هذا الموسم؟   

                                     original_answer  \
0  {'text': ['غلاف التوربين'], 'answer_start': [1...   
1  {'text': ['وجود البلاستيدات الخضراء المفقود'],...   
2  {'text': ['أصبحت فيه كليات الفنون التطبيقية جا...   
3  {'text': ['استشارات الطاقة العمالية في الأعمال...   
4    {'text': ['كاوان شورت'], 'answer_start': [216]}   

                                   generated_answers  
0  [المرفق بالسواكن هو أحد الأوتار التالية للشخص ...  
1  [تتضمن الدنا العِرضيّة نطاقات الدنا المُتِبَرِ...  
2  [حصل على جائزة نوبل في الآداب وتم إشهار قانون ...  
3  [6/ 13/ 1433 هـ, أجرى مجل

##RougeL

In [ ]:
! pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=34246a4d7ffa60e9b571f07b523f4e3f82f4cb6f4600dc800ca418a2af07523e
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [9]:
from transformers import AutoTokenizer
from rouge_score import rouge_scorer

# Load AraBERT tokenizer
model_name = 'aubmindlab/bert-base-arabertv02'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the scorer, passing in the AraBERT tokenizer
scorer = rouge_scorer.RougeScorer(['rougeL'], tokenizer=tokenizer)  # Pass the tokenizer object

# Example strings
pred_str = 'السلام عليكم كيف حالك'
label_str = 'السلام عليكم صديقي كيف حالك'

# Calculate ROUGE score directly using the input strings
scores = scorer.score(label_str, pred_str)  # Use original strings

for key in scores:
    print(f'{key}: {scores[key]}')

rougeL: Score(precision=1.0, recall=0.8333333333333334, fmeasure=0.9090909090909091)


In [15]:
import json
from transformers import AutoTokenizer
from rouge_score import rouge_scorer

# Load AraBERT tokenizer
model_name = 'aubmindlab/bert-base-arabertv02'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Function to load data from JSON file
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# Function to save labels to a new JSON file
def save_labels(output_data, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as file:
        json.dump(output_data, file, ensure_ascii=False, indent=4)

# Function to calculate ROUGE score and check hallucination
def check_hallucination(data):
    output_data = []  # List to store the labeled data

    for item in data:
        question_id = item['question_id']
        question = item['question']
        original_answer = item['original_answer']['text'][0]  # Get the first text of the original answer

        # Dictionary to store results for each question
        question_data = {
            "question_id": question_id,
            "question": question,
            "original_answer": item['original_answer'],  # Keep original answer as it is
            "generated_answers": []
        }

        for generated_answer in item['generated_answers']:
            # Calculate ROUGE-L F1 score
            scores = scorer.score(original_answer, generated_answer)
            rouge_l_f1 = scores['rougeL'].fmeasure

            # Check if hallucinated or not
            if rouge_l_f1 < 0.3:
                status = "Hallucinated"
            else:
                status = "Non-Hallucinated"

            # Append the labeled answer with F1 score to the question data
            question_data["generated_answers"].append({
                "text": generated_answer,
                "f1_score": round(rouge_l_f1, 2),  # Round to 2 decimal places
                "label": status
            })

        # Append the question data to the output list
        output_data.append(question_data)

    return output_data

# Load data from JSON file
input_file_path = '/content/drive/My Drive/generated_answers_Llama3.1-8b-xquad.json'  # Change this to the path of your input JSON file
data = load_data(input_file_path)

# Run the function and get labeled data
labeled_data = check_hallucination(data)

# Save the labeled data to a new JSON file
output_file_path = 'labeled_data.json'  # Output file name
save_labels(labeled_data, output_file_path)

print(f"Labeled data has been saved to {output_file_path}")


Labeled data has been saved to labeled_data.json
